[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Omerdan03/DanzDSTools/blob/master/SQL_tools.ipynb)

In [1]:
import mysql.connector as connector
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Don't forget the change those setting to your client and database 

In [2]:
host = 'localhost'
user = 'root'
password = ''
database = ''

In [3]:
def connect_to_mysql():
    """
    This function uses the configuration from config.py file and returns a connection the mysql. if stocks_db doesn't
    exists it create one according to stock_prices.sql file.
    :return: a connection at the stacks_db database:
    """
    con = connector.connect(host=host, user=user, password=password, database=database)
    return con

In [4]:
def send_statement(con, statement):
    """
    This function receives statement and connection and runs the statement. it return the return from that statement
    :param statement: String to be run on the db
    :param con: A connection object to the sql
    :return: the return from the db
    """
    cursor = con.cursor()
    cursor.execute(statement)
    return list(cursor)

In [5]:
def get_tables(con):
    """
    This function receives a connection and returns a list of the columns in that table
    :param con: A connection object to the sql
    :return: A list containing the tables in the db
    """
    quarry = f"SHOW TABLES"
    tables_tuple = send_statement(con, quarry)
    return [column[0] for column in tables_tuple]

In [6]:
def get_columns(con, table):
    """
    This function receives a connection and a table and returns a list of the columns in that table
    :param con: A connection object to the sql
    :param table: A table to collect the columns from
    :return: A list containing the columns in the list
    """
    quarry = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{table}'"
    columns_tuple = send_statement(con, quarry)
    return [column[0] for column in columns_tuple]

In [7]:
def get_table_as_df(con, table):
    """
    This function revices a connection and a table and returns a DataFrame with all the data from the table
    :param con: A connection object to the sql
    :param table: A table to collect the data from
    :return: A DataFrame from the table
    """
    columns = get_columns(con, table)
    quarry = f"SELECT * from {table}"
    arr = send_statement(con, quarry)
    return pd.DataFrame(arr, columns=columns)

In [8]:
def get_quarry(table, columns="*", limit=None, where=None, order=None, desc=False):
    """
    This function generates a SQL quarry accordion to the input parameters
    """
    if columns:
        columns_quarry = f"{', '.join(columns)}"
    else:
        columns_quarry = "*"
    if where:
        where_quarry = f"WHERE {where}"
    else:
        where_quarry = ""
    if limit:
        limit_quarry = f'LIMIT {limit}'
    else:
        limit_quarry = ""
    if order:
        order_quarry = f"ORDER BY {order}"
        if desc:
            order_quarry += " DESC"
    else:
        order_quarry = ""
    quarry = f"SELECT {columns_quarry} FROM {table} {where_quarry} {order_quarry} {limit_quarry};"
    return quarry

This is an example that takes the first table from the db and 

In [10]:
con = connect_to_mysql()
tables = get_tables(con)
get_table_as_df(con, tables[0])

full_name                gender id
0           933       Lewis Abernathy  M
1          2547        Andrew Adamson  M
2          2700          William Addy  M
3          2898           Seth Adkins  M
4          2925         Charles Adler  M
...         ...                   ... ..
1995     850088         Butch Wilhelm  M
1996     850089            Dalyn Chew  M
1997     850090  Celia Fushille-Burke  M
1998     850091         Mercedes Ngoh  M
1999     850092        Jennifer Jaffe  M

[2000 rows x 3 columns]